In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import keras
import os
import numpy as np
from keras.preprocessing import sequence


In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [ ]:
print(path_to_file)
text=open(path_to_file,'rb').read().decode(encoding='utf-8')

/root/.keras/datasets/shakespeare.txt


In [ ]:
print(len(text))


1115394


In [ ]:
vocab=sorted(set(text))

In [ ]:
char2idx={u:i for i,u in enumerate(vocab)}
idx2char=np.array(vocab)
print(idx2char)
def text_to_int(text):
  return np.array([char2idx[c] for c in text])

['\n' ' ' '!' '$' '&' "'" ',' '-' '.' '3' ':' ';' '?' 'A' 'B' 'C' 'D' 'E'
 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W'
 'X' 'Y' 'Z' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o'
 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']


In [ ]:
text_as_int=text_to_int(text)

In [ ]:
print(text[:13])
print(text_as_int[:13])

First Citizen
[18 47 56 57 58  1 15 47 58 47 64 43 52]


In [ ]:
def int_to_text(ints):
  try:
    ints=ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


In [ ]:
seq_length=100
examples_per_epoch=len(text)//(seq_length+1)
char_dataset=tf.data.Dataset.from_tensor_slices(text_as_int)

In [ ]:
sequences=char_dataset.batch(seq_length+1,drop_remainder=True)

In [ ]:
def split_input_target(chunk):
  input_text=chunk[:-1] #hell
  target_text=chunk[1:] #ello
  return input_text,target_text
dataset=sequences.map(split_input_target)

In [ ]:
for x,y in dataset.take(2):
  print(f"example\ninput:{int_to_text(x)}\noutput:{int_to_text(y)}")

example
input:First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You
output:irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 
example
input:are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 
output:re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [ ]:
batch_size=64
vocab_size=len(vocab)
embedding_dim=256
rnn_units=1024
buffer_size=10000
data=dataset.shuffle(buffer_size).batch(batch_size,drop_remainder=True)

In [ ]:
def build_model(vocab_size,embedding_dim,rnn_units,batch_size):
  model=tf.keras.Sequential(
      [
       tf.keras.layers.Embedding(vocab_size,embedding_dim,batch_input_shape=[batch_size,None]),
       tf.keras.layers.LSTM(rnn_units,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'),
       tf.keras.layers.Dense(vocab_size)
      ]
  )
  return model

In [ ]:
model=build_model(vocab_size,embedding_dim,rnn_units,batch_size)

In [ ]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (64, None, 256)           16640     
                                                                 
 lstm_7 (LSTM)               (64, None, 1024)          5246976   
                                                                 
 dense_7 (Dense)             (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [ ]:
for input_example_batch,target_example_batch in data.take(1):
  example_batch_predictions=model(input_example_batch)
  print(example_batch_predictions.shape) #(batch_size,sequence_length,vocab_size)

(64, 100, 65)


In [ ]:
print(len(example_batch_predictions))
#print(example_batch_predictions)

64


In [ ]:
pred=example_batch_predictions[0]
print(len(pred))
time_pred=pred[0]
print(len(time_pred))


100
65


In [ ]:
sampled_indices=tf.random.categorical(pred,num_samples=1)
sampled_indices=np.reshape(sampled_indices,(1,-1))[0]
predicted_chars=int_to_text(sampled_indices)
predicted_chars

"iPUmEks\n.rUGPRPz.Mb.\ncNFoIjaOYhj$?.$'p -afaErHRiEDALavBU'S Xm xaG3DiauUDckhEtxMVWJubppVL,UIr-iXYXoJ "

In [ ]:
#loss function
def loss(labels,logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels,logits,from_logits=True)

In [ ]:
model.compile(optimizer='adam',loss=loss)

In [ ]:
checkpoint_dir='./training_checkpoints'
checkpoint_prefix=os.path.join(checkpoint_dir,"ckpt_{epoch}")
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

In [ ]:
history=model.fit(data,epochs=50,callbacks=[checkpoint_callback])

Epoch 1/50
172/172 [==============================] - 14s 70ms/step - loss: 2.6639
Epoch 2/50
172/172 [==============================] - 14s 73ms/step - loss: 1.9574
Epoch 3/50
172/172 [==============================] - 13s 72ms/step - loss: 1.6962
Epoch 4/50
172/172 [==============================] - 13s 71ms/step - loss: 1.5548
Epoch 5/50
172/172 [==============================] - 13s 71ms/step - loss: 1.4699
Epoch 6/50
172/172 [==============================] - 14s 72ms/step - loss: 1.4122
Epoch 7/50
172/172 [==============================] - 13s 72ms/step - loss: 1.3677
Epoch 8/50
172/172 [==============================] - 14s 72ms/step - loss: 1.3306
Epoch 9/50
172/172 [==============================] - 13s 70ms/step - loss: 1.2990
Epoch 10/50
172/172 [==============================] - 13s 72ms/step - loss: 1.2694
Epoch 11/50
172/172 [==============================] - 14s 72ms/step - loss: 1.2410
Epoch 12/50
172/172 [==============================] - 13s 71ms/step - loss: 1.2136
E

In [ ]:
model=build_model(vocab_size,embedding_dim,rnn_units,batch_size=1)

In [ ]:
checkpoint_num=10
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1,None]))

In [ ]:
def generate_text(model,start_string):
  num_generate=800

  input_eval=[char2idx[s] for s in start_string]

  input_eval=tf.expand_dims(input_eval,0)

  text_generated=[]

  temperature=1.0

  model.reset_states()

  for i in range(num_generate):

    predictions=model(input_eval)

    predictions=tf.squeeze(predictions,0)

    predictions=predictions/temperature

    predicted_id=tf.random.categorical(predictions,num_samples=1)[-1,0].numpy()

    input_eval=tf.expand_dims([predicted_id],0)


    text_generated.append(idx2char[predicted_id])

  return (start_string+"".join(text_generated))

In [ ]:
inp=input("Type a starting string: ")
print(generate_text(model,inp))

Type a starting string: romeo
romeous heast is all of fury: shot to thy love:
and then we cannot, sir, that I am now gone;
Go, rall my hounds shall know.

KING RICHARD II:
No matter where is my fellow Tranio.

TRANIO:
Here comes no meal.
Good lady, good queen; I say, I'll rison for you:
Look, and the commanded morning lives of death,
But be at glim quit their own report.

Servant:
These detes mistrust its like the harm, time paul,
Thou canst demand was said is now he did
temper and tear of their sixteency, and fune more matter;
Prodigrace for your vantage. Be relegs and reason could I effected
To know our duty, and together with the heart.
Take thy commission.

ANGELO:
Now, good my lord,
Like almost time to do some four contented:
Methinks yourself no hatred, though not in prison.

ISABELLA:
O, I do fear thee, Tranio, be co
